# 02. 멀티암드 밴딧

멀티암드 밴딧
- n개의 손잡이가 달린 밴딧 문제
- 목표 : 가장 높은 보상을 제공하는 손잡이를 찾아 보상을 최대화
- 시간 의존성, 상태 의존성을 고려하지 않음

In [3]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [5]:
bandit_arms = [0.2, 0, -0.2, -2]
num_arms = len(bandit_arms)
def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [6]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_arms]))
output = tf.nn.softmax(weights)

reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)

responsible_output = tf.slice(output,action_holder,[1])
loss = -(tf.log(responsible_output)*reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

In [7]:
total_episodes = 1000
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        actions = sess.run(output)
        a = np.random.choice(actions,p=actions)
        action = np.argmax(actions == a)

        reward = pullBandit(bandit_arms[action])
        
        _,resp,ww = sess.run([update,responsible_output,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_arms) + " arms of the bandit: " + str(total_reward))
        i+=1
print("\nThe agent thinks arm " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 arms of the bandit: [1. 0. 0. 0.]
Running reward for the 4 arms of the bandit: [-2. -2.  0. 11.]
Running reward for the 4 arms of the bandit: [-7. -2.  4. 20.]
Running reward for the 4 arms of the bandit: [-9. -3.  4. 31.]
Running reward for the 4 arms of the bandit: [-10.   2.   1.  38.]
Running reward for the 4 arms of the bandit: [-15.   5.   4.  45.]
Running reward for the 4 arms of the bandit: [-14.   3.   9.  61.]
Running reward for the 4 arms of the bandit: [-14.   4.  10.  73.]
Running reward for the 4 arms of the bandit: [-11.  12.  11.  83.]
Running reward for the 4 arms of the bandit: [-12.   2.  15.  94.]
Running reward for the 4 arms of the bandit: [-15.  -5.  17. 104.]
Running reward for the 4 arms of the bandit: [-21.  -8.  19. 121.]
Running reward for the 4 arms of the bandit: [-29.  -5.  23. 130.]
Running reward for the 4 arms of the bandit: [-33.  -6.  23. 149.]
Running reward for the 4 arms of the bandit: [-39.  -6.  24. 160.]
Running reward 